In [12]:
import numpy as np
import baseline_model
from casadi import *
import do_mpc
import decentralized as dec
import util
import sim_drone_main
from baseline_model import baseline_drone_model
import itertools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
Q = np.eye(6)
R = np.eye(3)
n_agents = 3
n_inputs = 3
n_states = 6
Qf = Q*1e3

In [4]:
Qs = [Q] * n_agents
Rs = [R] * n_agents
Qfs = [Qf] * n_agents

In [5]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

x = model.set_variable(var_type='_x', var_name='x', shape=(6, 1))
u = model.set_variable(var_type='_u', var_name='u', shape=(3, 1))
print(f'Shape of x is {x.shape}')
print(f'Shape of u is {u.shape}')
#x = p_x,p_y,p_z,v_x,v_y,v_z
#u = theta, phi, tau
g = 9.81
model.set_rhs('x', vertcat(x[3], x[4], x[5], g*np.tan(u[0]), -g*np.tan(u[1]), u[2]-g))

Shape of x is (6, 1)
Shape of u is (3, 1)


In [6]:
theta_max = np.pi/6
phi_max = np.pi/6
tau_max = 5
v_max = 5

x_dims = [6,6,6]

Q = np.eye(n_states)*10
Qf = np.eye(n_states)*1e3
R = np.eye(n_inputs)

radius = 0.5
n_dims = [3,3,3]
episode=200
x_baseline_init, x_baseline_f = util.paper_setup_3_quads()

x_baseline1 = x_baseline_init

states_list = np.zeros((episode+1,9)) #positions of each drone
states_list[0,:] = np.array([x_baseline1[0],x_baseline1[1],x_baseline1[2],\
                        x_baseline1[6],x_baseline1[7],x_baseline1[8],\
                        x_baseline1[12],x_baseline1[13],x_baseline1[14]]).flatten()

In [7]:
# sim_drone_main.setup_baseline(x_baseline1, x_baseline_f, v_max, theta_max, phi_max, tau_max,\
#                     x_dims, Q, R, Qf, n_agents, n_dims, radius)

In [78]:
model_baseline = baseline_drone_model(x_baseline_f, x_dims, Q, R, Qf, n_agents, n_dims, radius)

Shape of x is (6, 1)
Shape of u is (3, 1)


In [50]:
model.set_expression('total_stage_cost',np.sum(total_stage_cost))
model.set_expression('total_terminal_cost',np.sum(total_terminal_cost))

SX(@1=1000, @2=0.67056, @3=0.3322, @4=0.589424, @5=0.59735, @6=0.0866769, @7=0.324466, @8=0.0406781, @9=0.0136367, @10=0.929077, @11=0.449056, @12=0.608274, @13=0.800826, @14=0.637524, @15=0.414727, @16=0.184916, @17=0.614452, @18=0.838018, @19=0.657215, (((((((((@1*(x_0-@2))*(x_0-@2))+((@1*(x_1-@3))*(x_1-@3)))+((@1*(x_2-@4))*(x_2-@4)))+((@1*(x_3-@5))*(x_3-@5)))+((@1*(x_4-@6))*(x_4-@6)))+((@1*(x_5-@7))*(x_5-@7)))+(((((((@1*(x_0-@8))*(x_0-@8))+((@1*(x_1-@9))*(x_1-@9)))+((@1*(x_2-@10))*(x_2-@10)))+((@1*(x_3-@11))*(x_3-@11)))+((@1*(x_4-@12))*(x_4-@12)))+((@1*(x_5-@13))*(x_5-@13))))+(((((((@1*(x_0-@14))*(x_0-@14))+((@1*(x_1-@15))*(x_1-@15)))+((@1*(x_2-@16))*(x_2-@16)))+((@1*(x_3-@17))*(x_3-@17)))+((@1*(x_4-@18))*(x_4-@18)))+((@1*(x_5-@19))*(x_5-@19)))))

In [52]:
radius = 0.5
n_dims = [3,3,3]

In [68]:
def compute_pairwise_distance(X, x_dims, n_d=2):
    """Compute the distance between each pair of agents"""
    assert len(set(x_dims)) == 1

    n_agents = len(x_dims)
    n_states = x_dims[0]

    if n_agents == 1:
        raise ValueError("Can't compute pairwise distance for one agent.")

    pair_inds = np.array(list(itertools.combinations(range(n_agents), 2)))
    X_agent = reshape(X,(n_agents, n_states))
    dX = X_agent[:n_d, pair_inds[:, 0]] - X_agent[:n_d, pair_inds[:, 1]]
    return np.linalg.norm(dX, axis=0).T

In [53]:
x_new = model.set_variable(var_type='_x', var_name='x_new', shape=(18, 1))

AssertionError: The variable name x_new for type _x already exists.

In [25]:
import itertools

In [81]:
distances = compute_pairwise_distance(x_baseline_f,x_dims,n_d=3)

pair_costs = SX(np.fmin(np.zeros(1), distances - radius) ** 2)

In [82]:
model.set_expression('total_prox_cost',pair_costs)

SX(@1=0, [@1, @1, @1])

In [65]:
SX.reshape?

Signature: SX.reshape(self, arg)
Docstring: <no docstring>
File:      ~/anaconda3/envs/Randy/lib/python3.10/site-packages/casadi/casadi.py
Type:      function
